# TODO: viz result from LOG FILE

# Step 0: Preparation
Before use this script, look into `README.md`
Ensure you have execute this command to get the data
```
./runexp.sh -s delay
```


In [33]:
import pickle
import pandas
import numpy as np

import header
import msgorganizer
import imp
imp.reload(header)
imp.reload(msgorganizer)

delay_dict = pickle.load(open(header.LOG_DELAY, 'rb'))
fparse = msgorganizer.FILEPARSER()
meta = fparse.return_metadict()   
print (f'data sample for expid={list(meta.keys())[0]}: {meta[list(meta.keys())[0]]}')

data sample for expid=0: {'fbasename': 'eng1_30_context_loc4_filesize', 'fps': 29.0, 'length': 12.0}


In [34]:
fieldName_list = ['recordLocation', 'patchSize', 'modelName', 'baseline', 'patchLocation', 'lightCondition', 'raw', 'expid', 'length', 'stall']


def parse_fbasename(expid, metaItem):
    
    fbasename0 = metaItem['fbasename']
    result = ['', '', '', '', '', '', '', '', '']
    fbasename = str(fbasename0)
    
    ##### SKIP if fbasename has '-', return empty array
    if fbasename0.find('-') >= 0:
        return []
    
    ###########STANDARDIZE-making all samples the same for processing############
    #replace all `-` with `_`
    fbasename = fbasename.replace('-', '_').replace('_hyperpri', '_hyper')
    
    #if there is no `_loc` (baselines), add loc0
    if fbasename.find('_loc') ==-1:
        fbasename = fbasename.replace('_filesize', '_loc0_filesize')
    
    #filter out `dark` string, mark the result as `dark`
    if fbasename.find('_dark_') >= 0:
        result[5] = 'dark'
        fbasename = fbasename.replace('_dark', '')
    else:
        result[5] = 'light'
        
    #add standard for fbasename without noise or clean
    if fbasename.find('_noise') >= 0:
        result[3] = 'noise'
        fbasename = fbasename.replace('_noise', '')
    elif fbasename.find('_clean') >= 0:
        result[3] = 'clean'
        fbasename = fbasename.replace('_clean', '')
    else:
        result[3] = 'standard'
    
    dat = fbasename.replace('_filesize', '').split('_')
    result[0] = dat[0]
    result[1] = dat[1]
    result[2] = dat[2]
    result[4] = dat[3]#note: mapping index 3 in input to index 4 in output, since we remove noise/clean/standard
    result[6] = fbasename0
    result[7] = expid
    result[8] = float(metaItem['length'])
    return result

expid = 0
parse_fbasename(expid, meta[expid])

['eng1',
 '30',
 'context',
 'standard',
 'loc4',
 'light',
 'eng1_30_context_loc4_filesize',
 0,
 12.0]

In [35]:
rawdat = []
for expid in delay_dict:
    #print (meta[expid], delay_dict[expid])
    datrow = parse_fbasename(expid, meta[expid])
    if len(datrow) == 0:
        continue
    datrow.append(delay_dict[expid])
    rawdat.append(datrow)
    print (meta[expid]['fbasename'], datrow)

eng1_30_context_loc4_filesize ['eng1', '30', 'context', 'standard', 'loc4', 'light', 'eng1_30_context_loc4_filesize', 0, 12.0, 6.1724137931034315]
eng1_30_context_loc7_filesize ['eng1', '30', 'context', 'standard', 'loc7', 'light', 'eng1_30_context_loc7_filesize', 1, 15.0, 5.58620689655171]
eng1_40_cheng_clean_filesize ['eng1', '40', 'cheng', 'clean', 'loc0', 'light', 'eng1_40_cheng_clean_filesize', 2, 12.0, 0.1724137931034483]
eng1_40_cheng_loc4_filesize ['eng1', '40', 'cheng', 'standard', 'loc4', 'light', 'eng1_40_cheng_loc4_filesize', 3, 14.0, 13.517241379310438]
eng1_40_cheng_loc7_filesize ['eng1', '40', 'cheng', 'standard', 'loc7', 'light', 'eng1_40_cheng_loc7_filesize', 4, 13.0, 12.310344827586274]
eng1_40_cheng_noise_loc4_filesize ['eng1', '40', 'cheng', 'noise', 'loc4', 'light', 'eng1_40_cheng_noise_loc4_filesize', 5, 15.0, 0.20689655172413796]
eng1_40_cheng_noise_loc7_filesize ['eng1', '40', 'cheng', 'noise', 'loc7', 'light', 'eng1_40_cheng_noise_loc7_filesize', 6, 12.0, 0.172

In [36]:
df = pandas.DataFrame(rawdat, columns=fieldName_list)
df = df[df['patchSize']=='40']
df[df['recordLocation']=='rh3']
#df[df['raw'] == 'rh3_40_cheng_noise_loc5_filesize']
len(df), len(rawdat), len(delay_dict), len(meta)

(146, 184, 187, 187)

# Stall result by model

In [55]:
set(df['modelName'].to_numpy())

{'cheng', 'context', 'hyper'}

### Use this code to get stall

In [65]:
#Result for `context` in bright light
modelName_list = set(df['modelName'].to_numpy())
print (f'Model names: {modelName_list}')
baselineName_list = set(df['baseline'].to_numpy())
recordLocation_list = set(df['recordLocation'].to_numpy())


result_detail = {}

for modelName in modelName_list:
    if modelName not in result_detail:
        result_detail[modelName] = {baseline:[] for baseline in baselineName_list}#{}
    for recordLocation in recordLocation_list:
            ds_clean    = df[(df['modelName']==modelName)&(df['recordLocation']==recordLocation)&(df['lightCondition']=='light')&(df['baseline']=='clean')]
            ds_noise    = df[(df['modelName']==modelName)&(df['recordLocation']==recordLocation)&(df['lightCondition']=='light')&(df['baseline']=='noise')]
            ds_standard = df[(df['modelName']==modelName)&(df['recordLocation']==recordLocation)&(df['lightCondition']=='light')&(df['baseline']=='standard')]
            
            stall_clean = (ds_clean['stall'].to_numpy()/ds_clean['length'].to_numpy()).mean() * 60
            stall_noise = (ds_noise['stall'].to_numpy()/ds_noise['length'].to_numpy()).mean() * 60
            stall_standard = (ds_standard['stall'].to_numpy()/ds_standard['length'].to_numpy()).mean() * 60
            #stall_noise = (ds_noise['stall'].to_numpy()/ds_noise['length'].to_numpy()).mean() * 60
            if stall_clean < stall_noise :
                print (modelName, recordLocation, ds_clean['raw'].to_numpy(), stall_clean, ds_noise['raw'].to_numpy(), stall_noise)
                print ("####")
            if np.isnan(stall_standard):
                continue
                
            result_detail[modelName]['clean'].append(stall_clean)
            result_detail[modelName]['noise'].append(stall_noise)
            result_detail[modelName]['standard'].append(stall_standard)

print ('####DETAILED RESULTS######')
print (result_detail)
            
result = {}
for modelName in modelName_list:
    result[modelName] = {baseline:[] for baseline in baselineName_list}#{}
    for baseline in baselineName_list:
        result[modelName][baseline] = np.mean(result_detail[modelName][baseline])
        
print ('####SUMMARIZED RESULT######')
print (result)

print ('#### PERCENTAGE DIFFERENCE#####')
for modelName in modelName_list:
    rclean, rnoise, rstandard = result[modelName]['clean'], result[modelName]['noise'], result[modelName]['standard']
    print (f'model: {modelName}, improve over noise: {rstandard/rnoise*100}, improve over clean: {rstandard/rclean}' )

Model names: {'hyper', 'context', 'cheng'}
hyper rh1 ['rh1_40_hyperpri_clean_filesize'] 41.22015915119361 ['rh1_40_hyperpri_noise_loc6_filesize'
 'rh1_40_hyperpri_noise_loc9_filesize'] 42.3626373626374
####
hyper eng1 ['eng1_40_hyper_clean_filesize'] 0.8620689655172414 ['eng1_40_hyper_noise_loc4_filesize' 'eng1_40_hyper_noise_loc7_filesize'] 0.9137931034482759
####
hyper jc ['jc_40_hyper_clean_filesize'] 44.689655172413964 ['jc_40_hyper_noise_loc6_filesize' 'jc_40_hyper_noise_loc9_filesize'] 45.935960591133174
####
hyper eng4 ['eng4_40_hyper_clean_filesize'] 5.793103448275864 ['eng4_40_hyper_noise_loc5_filesize' 'eng4_40_hyper_noise_loc6_filesize'
 'eng4_40_hyper_noise_loc8_filesize' 'eng4_40_hyper_noise_loc9_filesize'] 8.834312239484657
####
hyper eng2 ['eng2_40_hyper_clean_filesize'] 10.049261083743842 ['eng2_40_hyper_noise_loc6_filesize' 'eng2_40_hyper_noise_loc9_filesize'] 12.499999999999996
####
hyper rh2 ['rh2_40_hyper_clean_filesize'] 37.40053050397868 ['rh2_40_hyper_noise_loc4_

/tmp/ipykernel_46777/3268192914.py:20: RuntimeWarning: Mean of empty slice.
  stall_standard = (ds_standard['stall'].to_numpy()/ds_standard['length'].to_numpy()).mean() * 60
/apps/mambaforge/envs/env_hmd_attack/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [63]:
baselineName_list

{'clean', 'noise', 'standard'}

### The code below has problem, ignore it

In [45]:
#Result for `context` in bright light
modelName_list = set(df['modelName'].to_numpy())
print (f'Model names: {modelName_list}')
baselineName_list = ['standard', 'clean', 'noise']

result = {}
for modelName in modelName_list:
    if modelName not in result:
        result[modelName] = {}
    for baselineName in baselineName_list:
        ds = df[(df['modelName']==modelName)&(df['lightCondition']=='light')&(df['baseline']==baselineName)]
        stall_list = ds['stall'].to_numpy()
        length_list = ds['length'].to_numpy()
        result[modelName][baselineName] = ((stall_list/length_list) * 60).mean()
        print (ds)
        print ("#####")

print (result)
        
# for k in result:
#     dat = result[k]
#     print (k, dat['standard']/dat['noise'] * 100, dat['standard']/dat['clean'] * 100)

Model names: {'hyper', 'context', 'cheng'}
    recordLocation patchSize modelName  baseline patchLocation lightCondition   
18            eng1        40     hyper  standard          loc4          light  \
19            eng1        40     hyper  standard          loc7          light   
41            eng2        40     hyper  standard          loc6          light   
42            eng2        40     hyper  standard          loc9          light   
65            eng3        40     hyper  standard          loc4          light   
66            eng3        40     hyper  standard          loc6          light   
67            eng3        40     hyper  standard          loc7          light   
68            eng3        40     hyper  standard          loc9          light   
98            eng4        40     hyper  standard          loc5          light   
99            eng4        40     hyper  standard          loc6          light   
100           eng4        40     hyper  standard          loc8    

In [38]:
#NOTE: reported location in the email 'jc', 'eng1', 'eng2', 'eng3', 'rh1'
#location for best result: 'jc', 'eng1', 'eng2', 'eng3'

In [39]:
for modelName in modelName_list:
    for baselineName in baselineName_list:
        ds = df[(df['modelName']==modelName)&(df['lightCondition']=='light')&(df['baseline']==baselineName)]
        print (modelName, baselineName, set(ds['recordLocation'].to_numpy()))

hyper standard {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
hyper clean {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
hyper noise {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
context standard {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2'}
context clean {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
context noise {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
cheng standard {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
cheng clean {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}
cheng noise {'rh1', 'eng1', 'jc', 'eng4', 'eng2', 'eng3', 'rh2', 'rh3'}


# Lightning results

In [40]:
MODELNAME_CONTEXT = 'context'
lightCondition_list = ['light', 'dark']


result = {}
for lightCondition in lightCondition_list:
    if lightCondition not in result:
        result[lightCondition] = {}
    for baselineName in baselineName_list:
        stall = df[(df['modelName']==MODELNAME_CONTEXT)&(df['lightCondition']==lightCondition)&(df['baseline']==baselineName)]['stall'].to_numpy().mean()
        result[lightCondition][baselineName] = stall

print (result)
        
for k in result:
    dat = result[k]
    print (k, dat['standard']/dat['noise'] * 100, dat['standard']/dat['clean'] * 100)

{'light': {'standard': 10.785440613026857, 'clean': 4.094827586206897, 'noise': 3.8218390804597706}, 'dark': {'standard': 11.922413793103505, 'clean': 0.6034482758620687, 'noise': 0.8448275862068968}}
light 282.2055137844621 263.39181286549797
dark 1411.2244897959247 1975.714285714296


In [41]:
MODELNAME_CONTEXT = 'context'
recordLocation_list = set(df['recordLocation'].to_numpy())

result = {}
for recordLocation in recordLocation_list:
    stall = df[(df['recordLocation']==recordLocation)&(df['baseline']=='standard')]['stall'].mean()
    result[recordLocation] = stall

print (result)
        


{'rh1': 11.724137931034535, 'eng1': 11.75431034482764, 'jc': 13.62068965517251, 'eng4': 10.53735632183911, 'eng2': 12.254310344827651, 'eng3': 11.037356321839125, 'rh2': 12.10344827586213, 'rh3': 7.577586206896564}


# Debug why noise < clean